In [202]:
import pandas as pd
old_df = pd.read_csv('dannye_3mes.csv', sep=';', encoding='cp1252')
old_df.columns = 'Кредит (актив);Вклад (пассив);Дата погашения;Срок погашения, дней;Ставка, %;[ОТВЕТ] Номер группы;Пример группировки (для первых 20 сделок);'.split(';')
new_df = old_df.groupby(by=['Ставка, %', 'Срок погашения, дней'], as_index=False).sum()
new_df['s'] = new_df['Кредит (актив)'] - new_df['Вклад (пассив)']
r, t, s = new_df['Ставка, %'], new_df['Срок погашения, дней'], new_df['s']
d = new_df.rename(columns={'Ставка, %':'r','Срок погашения, дней':'t', '[ОТВЕТ] Номер группы': 'n'}).loc[:, ('r', 't', 's', 'n')]

In [203]:
dr = 0.15 / 2
dt = 30 / 2
c = 0
was = []
for num_1, j in enumerate(zip(d['r'], d['t'], d['s'])): 
    if num_1 in was:
        continue
    r, t, s = j 
    was.append(num_1)
    d['n'][num_1] = c
    new_df['[ОТВЕТ] Номер группы'][num_1]  = c
    indexes = d[(abs(d.r - r ) <= dr) & (abs(d.t - t) <= dt)].index
    for i in indexes:
        if i not in was :
            was.append(i)
            d['n'][i] = c
            new_df['[ОТВЕТ] Номер группы'][i] = c
    c += 1 

/tmp/ipykernel_2843/325260508.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['n'][num_1] = c
/tmp/ipykernel_2843/325260508.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['[ОТВЕТ] Номер группы'][num_1]  = c
/tmp/ipykernel_2843/325260508.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['n'][i] = c
/tmp/ipykernel_2843/325260508.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [221]:
temp = d.groupby(by='n', as_index=False).agg(['sum', 'mean', 'max'])
nd = pd.DataFrame({'r': temp['r']['max'], 't': temp['t']['max'], 's': temp['s']['sum']})
nd

,r,t,s
n,,,
0.0,0.9050,45,3.115000e+08
1.0,0.9340,77,5.000000e+07
2.0,0.9745,37,-5.000000e+07
3.0,1.0300,63,3.200000e+07
4.0,1.0560,65,-5.000000e+07
5.0,1.2360,57,7.600000e+07
6.0,1.2450,74,1.000000e+08
7.0,1.4050,58,-1.780000e+08
8.0,1.3830,76,2.550000e+07


In [222]:
def rwa(t, a):
    p = 0 
    if a <= 0:
        p, a = a, p 
    w_gross = 0.05 * (2 if t < 31 else 1)
    w_net = 0.2 * (2 if t < 31 else 1)
    return (a + p) * w_gross + abs(a - p) * w_net

nd['rwa'] = nd.apply(lambda x: rwa(x['t'], x['s']), axis=1)
print("RWA после:\n", nd['rwa'].sum())
print("RWA до:\n", nd['s'].sum())

RWA после:
 894095352.987
RWA до:
 2032472987.1800003


In [220]:
d.to_csv('промежуточный_результат.csv')
for r, t, N in zip(new_df['Ставка, %'], new_df['Срок погашения, дней'],
                   new_df['[ОТВЕТ] Номер группы']):
    indexes = old_df[(old_df['Срок погашения, дней'] == t) & (old_df['Ставка, %'] == r)].index
    for i in indexes:
        old_df['[ОТВЕТ] Номер группы'][i] = N
old_df.to_csv('answer.csv')

/tmp/ipykernel_2843/3093328590.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_df['[ОТВЕТ] Номер группы'][i] = N


In [226]:
pd.read_csv('answer.csv')['[ОТВЕТ] Номер группы'].value_counts()

13.0    727
16.0    392
14.0    377
18.0    266
15.0    216
19.0    132
12.0      9
25.0      5
23.0      4
26.0      3
22.0      3
32.0      3
11.0      3
28.0      2
35.0      2
10.0      2
7.0       2
9.0       2
20.0      2
17.0      2
21.0      2
3.0       2
0.0       2
4.0       1
8.0       1
37.0      1
31.0      1
36.0      1
1.0       1
24.0      1
27.0      1
2.0       1
6.0       1
34.0      1
29.0      1
33.0      1
5.0       1
30.0      1
Name: [ОТВЕТ] Номер группы, dtype: int64